# `AStream` Online Training

**[THIS IS WORK IN PROGRESS]**

This notebook performs online training of the **appearance stream parent model** on the **car-shadow** sequence, so make sure you've run the [`AStream` Offline Training](astream_offline_training.ipynb) notebook before running this one.


The online training of the `AStream` network is done by finetuning the parent model **on the first frame** of the video sequence. This is the only frame for which a mask is provided. It is augmented using scaling and vertical flipping. The network is trained for 500 iterations using the same training parameters as during offline training, except that deep supervision is disabled.

![](img/osvos_child.png)

To monitor training, run:
```
tensorboard --logdir E:\repos\tf-video-seg\tfvos\models\astream_car-shadow
http://localhost:6006
```

In [1]:
"""
astream_online_training.ipynb

AStream online trainer

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Based on:
  - https://github.com/scaelles/OSVOS-TensorFlow/blob/master/osvos_parent_demo.py
    Written by Sergi Caelles (scaelles@vision.ee.ethz.ch)
    This file is part of the OSVOS paper presented in:
      Sergi Caelles, Kevis-Kokitsi Maninis, Jordi Pont-Tuset, Laura Leal-Taixe, Daniel Cremers, Luc Van Gool
      One-Shot Video Object Segmentation
      CVPR 2017
    Unknown code license
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
from PIL import Image
import numpy as np
import tensorflow as tf
slim = tf.contrib.slim
import matplotlib.pyplot as plt

In [2]:
# Import model files
import model
import datasets

## Configuration

In [3]:
# Model paths
seq_name = "car-shadow"
segnet_stream = 'astream'
parent_path = 'models/' + segnet_stream + '_parent/' + segnet_stream + '_parent.ckpt-50000'
ckpt_name = segnet_stream + '_' + seq_name
logs_path = 'models/' + ckpt_name

# Online training parameters
gpu_id = 0
max_training_iters = 500
learning_rate = 1e-8
save_step = max_training_iters
side_supervision = 3
display_step = 10

## Dataset load

In [4]:
# Load the DAVIS 2016 sequence
options = datasets._DEFAULT_DAVIS16_OPTIONS
options['use_cache'] = False
options['data_aug'] = True
# Set the following to wherever you have downloaded the DAVIS 2016 dataset
dataset_root = 'E:/datasets/davis2016/' if sys.platform.startswith("win") else '/media/EDrive/datasets/davis2016/'
test_frames = sorted(os.listdir(dataset_root + 'JPEGImages/480p/' + seq_name))
test_imgs = ['JPEGImages/480p/' + seq_name + '/' + frame for frame in test_frames]
train_imgs = ['JPEGImages/480p/' + seq_name + '/' + '00000.jpg ' + 'Annotations/480p/' + seq_name + '/' + '00000.png']
dataset = datasets.davis16(train_imgs, test_imgs, dataset_root, options)

Initializing dataset...
['JPEGImages/480p/car-shadow/00000.jpg Annotations/480p/car-shadow/00000.png']
Loading training images and masks...


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 43.38it/s]


...done loading training images and masks.
Performing scaling data augmentation on frames/masks...


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 199.45it/s]


... done performing scaling data augmentation on frames/masks.
Performing flipping data augmentation on frames/masks...


...

...done converting images and masks to numpy arrays.
Computing optical flows and warping masks...


video: 100%|█████████████████████████████████████| 6/6 [00:00<00:00, 854.67it/s]


...done with computing optical flows and warping masks.
Loading testing images...


100%|██████████████████████████████████████████| 40/40 [00:00<00:00, 142.74it/s]


...done loading testing images.
...done initializing Dataset


In [5]:
# Display dataset configuration
dataset.print_config()


Configuration:
  in_memory            True
  data_aug             True
  use_cache            False
  use_optical_flow     True
  use_warped_masks     True
  use_bboxes           True
  optical_flow_mgr     pyflow


## Online Training

In [6]:
# Finetune this branch of the binary segmentation network
with tf.Graph().as_default():
    with tf.device('/gpu:' + str(gpu_id)):
        global_step = tf.Variable(0, name='global_step', trainable=False)
        model.train_finetune(dataset, parent_path, side_supervision, learning_rate, logs_path, max_training_iters,
                             save_step, display_step, global_step, segnet_stream, iter_mean_grad=1, ckpt_name=ckpt_name)


Network Layers:
   name = astream/conv1/conv1_1/Relu:0, shape = (1, ?, ?, 64)
   name = astream/conv1/conv1_2/Relu:0, shape = (1, ?, ?, 64)
   name = astream/pool1/MaxPool:0, shape = (1, ?, ?, 64)
   name = astream/conv2/conv2_1/Relu:0, shape = (1, ?, ?, 128)
   name = astream/conv2/conv2_2/Relu:0, shape = (1, ?, ?, 128)
   name = astream/pool2/MaxPool:0, shape = (1, ?, ?, 128)
   name = astream/conv3/conv3_1/Relu:0, shape = (1, ?, ?, 256)
   name = astream/conv3/conv3_2/Relu:0, shape = (1, ?, ?, 256)
   name = astream/conv3/conv3_3/Relu:0, shape = (1, ?, ?, 256)
   name = astream/pool3/MaxPool:0, shape = (1, ?, ?, 256)
   name = astream/conv4/conv4_1/Relu:0, shape = (1, ?, ?, 512)
   name = astream/conv4/conv4_2/Relu:0, shape = (1, ?, ?, 512)
   name = astream/conv4/conv4_3/Relu:0, shape = (1, ?, ?, 512)
   name = astream/pool4/MaxPool:0, shape = (1, ?, ?, 512)
   name = astream/conv5/conv5_1/Relu:0, shape = (1, ?, ?, 512)
   name = astream/conv5/conv5_2/Relu:0, shape = (1, ?, ?, 512)

2018-02-01 17:11:40.797934 Iter 350: Training Loss = 186.4202
2018-02-01 17:11:42.431737 Iter 360: Training Loss = 126.5741
2018-02-01 17:11:44.118658 Iter 370: Training Loss = 211.3812
2018-02-01 17:11:45.737657 Iter 380: Training Loss = 124.4006
2018-02-01 17:11:47.489738 Iter 390: Training Loss = 123.9258
2018-02-01 17:11:49.195299 Iter 400: Training Loss = 118.8080
2018-02-01 17:11:50.920734 Iter 410: Training Loss = 205.2335
2018-02-01 17:11:52.493569 Iter 420: Training Loss = 203.9605
2018-02-01 17:11:54.169745 Iter 430: Training Loss = 117.3316
2018-02-01 17:11:55.898870 Iter 440: Training Loss = 176.2258
2018-02-01 17:11:57.522833 Iter 450: Training Loss = 200.6675
2018-02-01 17:11:59.182796 Iter 460: Training Loss = 199.5491
2018-02-01 17:12:00.815137 Iter 470: Training Loss = 173.7556
2018-02-01 17:12:02.486714 Iter 480: Training Loss = 175.8937
2018-02-01 17:12:04.092579 Iter 490: Training Loss = 114.4267
2018-02-01 17:12:05.686570 Iter 500: Training Loss = 117.2427
INFO:ten

## Training losses & learning rate
You should get training curves similar to the following:
![](img/astream_car-shadow_main_loss.png)
![](img/astream_car-shadow_total_loss.png)
![](img/astream_car-shadow_learning_rate.png)

## Testing

In [ ]:
# Result path (if you want to check how well this branch is doing on its own)
result_path = dataset_folder + 'Results/Segmentations/480p/' + ckpt_name

# Test this branch of the network
with tf.Graph().as_default():
    with tf.device('/gpu:' + str(gpu_id)):
        ckpt_path = logs_path + '/' + ckpt_name + '.ckpt-' + str(max_training_iters))
        model.test(dataset, ckpt_path, result_path)

Log output should be similar to the following:
```
INFO:tensorflow:Restoring parameters from models\car-shadow_new\car-shadow_new.ckpt-500
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00000.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00001.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00002.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00003.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00004.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00005.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00006.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00007.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00008.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00009.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00010.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00011.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00012.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00013.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00014.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00015.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00016.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00017.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00018.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00019.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00020.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00021.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00022.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00023.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00024.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00025.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00026.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00027.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00028.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00029.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00030.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00031.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00032.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00033.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00034.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00035.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00036.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00037.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00038.png
Saving E:/datasets/davis2016\Results\Segmentations\480p\OSVOS\car-shadow_new\00039.png
```

## Visual evaluation

Let's load the original images and their predicted masks to get an idea of how this branch of the network is doing. Note that the first mask is displayed in red overlay, as it is given to us. The predicted masks are displayed using a green overlay.

In [ ]:
# Load results
frames = []
predicted_masks=[]
for test_frame in test_frames:
    frame_num = test_frame.split('.')[0]
    frame = np.array(Image.open(dataset_root + 'JPEGImages/480p/' + seq_name + '/' + test_frame))
    predicted_mask = np.array(Image.open(result_path + frame_num +'.png'))
    frames.append(frame)
    predicted_masks.append(predicted_mask)

In [ ]:
# Overlay the masks on top of the frames
frames_with_predictions = visualize.overlay_frames_with_predictions(frames, predicted_masks')

### Display individual frames

In [ ]:
visualize.display_images(frames_with_predictions)

### Display results as a video clip

In [ ]:
# Set path to video clips
video_clip_folder = dataset_root + 'clips/'
video_clip = video_clip_folder + ckpt_name + '.mp4'

# Combine images in a video clip
visualize.make_clip(video_clip, frames_with_predictions)

In [ ]:
# Display video
visualize.show_clip(video_clip)